In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unstloh", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = 'prompt',
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
      warmup_steps = 5,
      num_train_epochs = 10, # For longer training runs!
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
device = "cuda:0"

query = ''' ترجم الي اللهجه المصريه: "After graduation I came back to Egypt to study in college, and now I have been living in Egypt for over 5 years and 3 months, I love Egypt."'''


prompt_template = """<start_of_turn>user يوجد أدناه تعليمات تصف المهمة. اكتب الرد الذي يكمل الطلب بشكل مناسب.\n\n{query}<end_of_turn>\n<start_of_turn>model"""

prompt = prompt_template.format(query=query)

inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

model_inputs = inputs.to(device)

In [ ]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id)

In [ ]:
if True: model.save_pretrained_merged("English2Egyptian", tokenizer, save_method = "merged_16bit",)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "English2Egyptian", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/English2Egyptian /content/drive/MyDrive/